# AMAL - TP 7
# Réseaux convolutifs (1D)


### 简介（简要说明，参考课程）

卷积神经网络（CNN，全称为 "Convolutional Neural Network"）是一种主要用于分类序列（1D）、图像（2D）或一般呈现对称性数据的架构。在最常见的情况下，这些对称性使得研究某个个体的方式对平移操作保持不变。比如，在图像中检测一张嘴的位置与这张嘴在图像中的具体位置无关；对于文本来说，判断某段内容提到的是一只红棕色的动物，也与该段内容在文本中的具体位置无关。

### 1 情感检测

在本次实验中，我们将使用一维卷积神经网络（1D CNN）对序列进行分类实验，但其工作原理是通用的。

- ### 问题 1

请使用 SentencePiece 进行可变分词（为此使用提供的脚本 `tp8_preprocess.py`）。立即启动该过程，并指定所需的词汇表大小（这可能需要较长时间）。

在一维卷积中（如图 1 所示），可以将卷积视为一种线性变换，作用于序列中大小为 **kernel_size** 的元素组（图中为 3 个元素），并以 **stride** 为步长（图中为 1）重复进行移动。

每次线性变换会将一个大小为 **kernel_size × in_channels** 的向量转换为一个大小为 **out_channels** 的向量，其中 **in_channels** 是输入数据的维度，**out_channels** 是输出的维度。这个输出维度可以看作是大小为 **kernel_size** 的序列上所使用的滤波器数量。在本次实验中，您将使用 PyTorch 提供的模块 **torch.nn.Conv1d**，该模块高效实现了一维卷积操作。

在卷积神经网络（CNNs）中，另一个常见的层是池化操作器（pooling）。其工作原理在大部分方面与卷积操作相似，但不同之处在于使用的转换函数。对于池化操作，它对输入向量的每个分量进行聚合。例如，最大池化（max-pooling）返回的是最大值（可以使用 **torch.nn.MaxPool1d** 实现）。

卷积操作与最大池化的交互方式如下：

1. **卷积**用于在短序列（例如，3 个单词的序列）中检测一组模式（如“一个好电影”“非常好电影”或“一个令人失望的电影”）。
2. **最大池化**通过聚合操作对更大的子序列中的过滤器检测结果进行总结。例如，大小为 5、步长为 2 的最大池化会总结子序列 (y1, ..., y5)，然后是 (y3, ..., y7)，以此类推。

这种操作可以重复多次，以便逐步检测出更大、更复杂的模式。例如，在第一层卷积中可以检测出“我非常喜欢”“我极其喜欢”“不太喜欢”以及“这部电影”“这位演员”（4 个滤波器）。在第二层卷积中，这些信息会被组合起来，检测出对电影或演员的评价。

**最后一层**通常是一个全局最大池化（针对输出的每个分量），之后连接一个线性分类器。

### 数据说明
实验中使用的数据集来自 **Sentiment140**。类别为：
- 0 = 消极 (negative)，
- 2 = 中性 (neutral)，
- 4 = 积极 (positive)。  
在预处理后的数据中，这些类别已被转换为 0、1 和 2。

- ### 问题 2

在检查数据类型后，请设计一些卷积网络的架构，并比较它们的分类准确率。您需要先评估一个简单的基线算法（该算法始终返回数据中占多数的类别）的性能，并将这些架构的相对性能与该基线模型进行比较。

### 2 研究 CNN 的行为

为了研究 CNN 的具体作用，我们将关注**全局最大池化之前的最后一层**；尤其是寻找训练集中能最大激活每个输出滤波器的子序列。

#### 示例  
首先，需要确定每个输出对应文本中的位置。例如：  
- 假设卷积核大小为 3，步长为 1，那么第 1 个输出对应文本中第 1 到第 3 个位置，第 2 个输出对应文本中第 2 到第 4 个位置，依此类推。  
- 如果加入一个最大池化操作（池化核大小为 2，步长为 2），那么第 1 个输出对应文本中第 1 到第 4 个位置，第 2 个输出对应文本中第 3 到第 6 个位置，依此类推。

#### 推广  
为了通用化，我们将第 $i$ 次操作（卷积/池化）视为仅由其**核宽度** $w_i$（kernel width）和**步长** $s_i$（stride）定义。我们关心的是描述从第 0 层到第 $i-1$ 层所有变换的两个值：  
- **输入长度** $W_{i-1}$：对应第 $i-1$ 层的单个输出。  
- **输入步长** $S_{i-1}$：对应第 $i-1$ 层输出中的单位步长在输入中的移动距离。

- ### 问题 3

给出递推公式，用于在已知 $W_i$、$S_i$ 的情况下，根据 $w_{i+1}$ 和 $s_{i+1}$ 确定 $W_{i+1}$ 和 $S_{i+1}$。

递推公式如下：  
- $W_{i+1} = \left\lfloor \frac{W_i - w_{i+1}}{s_{i+1}} \right\rfloor + 1$ 
  表示第 $i+1$ 层输出的长度，取决于第 $i$ 层输入的长度 $W_i$、核大小 $w_{i+1}$ 和步长 $s_{i+1}$。  
- $S_{i+1} = S_i \cdot s_{i+1}$  
  表示第 $i+1$ 层输出中单位步长在第 $i$ 层输入中的移动距离。

- ### 问题 4

设 $(y_1, \dots, y_L)$ 为 CNN 的输出。在完成这些计算后，给出一个公式，用于在已知输出 $y_j$ 的位置 $j$ 时，确定其在输入序列中的对应索引范围。

公式如下：

- **起始索引**：  
  $ \text{start\_index}_j = (j - 1) \cdot S_n + 1 $  

- **结束索引**：  
  $ \text{end\_index}_j = (j - 1) \cdot S_n + W_n $ 

其中：  
- $S_n$ 是从输入到最后一层输出的累计步长。  
- $W_n$ 是最后一层卷积核的大小（或池化操作的核大小）。  

这个公式用于从输出位置 $j$ 确定其在输入序列中的起始和结束索引范围 $[\text{start\_index}_j, \text{end\_index}_j]$。

- ### 问题 5

最后，遍历训练集数据，找到能够最大激活每个输出特征的子序列。